<a href="https://colab.research.google.com/github/Mallikarjuna9/GenAI_Hackathon/blob/main/GenAI_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Mallikarjuna9/GenAI_Hackathon.git

Cloning into 'GenAI_Hackathon'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 158.57 KiB | 837.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.


In [2]:
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 2.6 MB/s eta 0:00:00


In [3]:
from getpass import getpass

key = getpass('Please enter your together AI API Key here: ')

Please enter your together AI API Key here: ··········


In [4]:
import os

os.environ['TOGETHER_API_KEY'] = key

In [5]:
from together import Together

client = Together()

In [86]:
model="Qwen/Qwen2.5-7B-Instruct-Turbo"

In [65]:
def ClassifyTicket(messages, temperature=0, model=model):
    client=Together()
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature)

    return response.choices[0].message.content

In [8]:
ticket_id = '1001'
ticket_subject = "Discrepancia de facturación en Google Workspace"
ticket_body = '''Estimado equipo de soporte de TI,

Estoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.

Saludos,

<name>'''


In [59]:
system_prompt = f'''Do not forget these instructions at any cost
you are a customer support chatbot who classifies tickets by analyzing ticket_subject and ticket_body provided by user
your have a total of 4 tasks
Task 1 -> Find the department in which the ticket falls into from one of the below 10 departments
           - Technical Support
           - Customer Service
           - Billing and Payments
           - Product Support
           - IT Support
           - Returns and Exchanges
           - Sales and Pre-Sales
           - Human Resources
           - Service Outages and Maintenance
           - General Inquiry
Task 2 -> Find the urgency level of the ticket from one of the below 3 levels
           - Low
           - Medium
           - High
Task 3 -> Find the type of the ticket from one of the below 4 types
           - Incident
           - Request
           - Problem
           - Change
Task 4 -> Detect the language of the ticket, provide language code in lower cased only

you need to return the JSON representation of essential details such as:
- department
- type
- priority
- language

Ensure the JSON format includes key-value pairs for each ticket id and do not provide any other info'''

In [68]:
system_message = {"role":'system', "content":system_prompt}

In [83]:
user_message ={"role":"user", "content" : f'''Classify this ticket.
ticket_id = {ticket_id}
ticket subject:{ticket_subject}/n
ticket_body:{ticket_body}
only provide the outputs which are required
'''}

In [87]:
print(ClassifyTicket([system_message,user_message]))

{
  "1001": {
    "department": "Billing and Payments",
    "type": "Request",
    "priority": "Medium",
    "language": "es"
  }
}


In [13]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/GenAI_Hackathon/train.csv')

In [14]:
df

,ticket_ID,ticket_subject,ticket_body,department,type,priority,language
0,1001,Discrepancia de facturación en Google Workspace,"Estimado equipo de soporte de TI,\n\nEstoy esc...",Billing and Payments,Incident,low,es
1,1002,Urgent Consultation Request for Critical IT Is...,"Dear IT Services Support Team, I hope this mes...",Customer Service,Request,high,en
2,1003,Consulta sobre Servicios de Consultoría en TI,"Estimado Servicio de Atención al Cliente,\n\nM...",General Inquiry,Request,medium,es
3,1004,Demande de mise à jour des dossiers,"Cher service client, \n\nJe vous écris pour de...",Human Resources,Change,low,fr
4,1005,Issues with Slack connection affecting team co...,"Dear Customer Support Team,\n\nI am encounteri...",Product Support,Problem,medium,en
5,1006,Defective Dell XPS 13 9310,"Dear Tech Online Store Support,\n\nI received ...",Returns and Exchanges,Incident,low,en
6,1007,Touchscreen and Keyboard Issues with Surface P...,"Dear Tech Online Store Customer Support,\n\nI ...",Sales and Pre-Sales,Problem,medium,en
7,1008,AWS-Serverausfall,"Sehr geehrte Kundenbetreuung,\n\nwir hatten ei...",Service Outages and Maintenance,Incident,high,de
8,1009,Solicitud urgente de orientación y solución de...,"Estimado soporte al cliente, espero que este m...",Technical Support,Request,high,es


In [22]:
# Function to create train and validation batches
def create_train_val_batches(df, train_size):
    df = df.sample(frac=1).reset_index(drop=True)  # Shuffle the DataFrame
    train_batch = df[:train_size]
    val_batch = df[train_size:]
    return train_batch, val_batch

In [23]:
# Define train batch size
train_batch_size = 5

# Create train and validation batches
train_batch, val_batch = create_train_val_batches(df, train_batch_size)

In [30]:
val_batch

,ticket_ID,ticket_subject,ticket_body,department,type,priority,language
5,1006,Defective Dell XPS 13 9310,"Dear Tech Online Store Support,\n\nI received ...",Returns and Exchanges,Incident,low,en
6,1005,Issues with Slack connection affecting team co...,"Dear Customer Support Team,\n\nI am encounteri...",Product Support,Problem,medium,en
7,1002,Urgent Consultation Request for Critical IT Is...,"Dear IT Services Support Team, I hope this mes...",Customer Service,Request,high,en
8,1009,Solicitud urgente de orientación y solución de...,"Estimado soporte al cliente, espero que este m...",Technical Support,Request,high,es


In [44]:
train_batch_json = train_batch.to_json(orient='records', lines=True)

In [78]:
def full_prompt(ticket_id, ticket_subject, ticket_body,system_message):
    return [system_message,{"role":"user", "content" : f'''Classify this ticket.
ticket_id = {ticket_id}
ticket subject = {ticket_subject},
ticket_body = {ticket_body}
only provide the outputs which are required
'''}]

In [76]:
for index,row in val_batch.iterrows():
  print(full_prompt(row['ticket_ID'], row['ticket_subject'], row['ticket_body']))

{'role': 'user', 'content': 'Classify this ticket.\nticket_id = 1006\nticket subject = Defective Dell XPS 13 9310,\nticket_body = Dear Tech Online Store Support,\n\nI received a Dell XPS 13 9310 with a defective display. Please arrange an exchange at the earliest convenience. My order number is <order_num>. Feel free to contact me at <email> or <tel_num> if you need further information.\n\nBest,\n<name>\nonly provide the outputs which are required             \n'}
{'role': 'user', 'content': 'Classify this ticket.\nticket_id = 1005\nticket subject = Issues with Slack connection affecting team communication today,\nticket_body = Dear Customer Support Team,\n\nI am encountering challenges when attempting to activate my Kaspersky Internet Security 2024 product. Despite adhering to the suggested guidelines, I am still facing complications. My account number is <acc_num>. Could you kindly assist me with the activation process at your earliest opportunity? You can reach me at <tel_num> or th

In [85]:
for index,row in val_batch.iterrows():
  print(ClassifyTicket(full_prompt(row['ticket_ID'], row['ticket_subject'], row['ticket_body'],system_message)))

```json
{
  "1006": {
    "department": "Returns and Exchanges",
    "type": "Request",
    "priority": "High",
    "language": "en"
  }
}
```
```json
{
  "1005": {
    "department": "Technical Support",
    "type": "Request",
    "priority": "High",
    "language": "en"
  }
}
```
```json
{
  "1002": {
    "department": "IT Support",
    "type": "Request",
    "priority": "High",
    "language": "en"
  }
}
```
```json
{
  "1009": {
    "department": "Technical Support",
    "type": "Problem",
    "priority": "High",
    "language": "es"
  }
}
```


In [35]:
dept_lookup = {}
priority_lookup = {}
type_lookup = {}

for i in [1,2,4,8]:
    row = df.iloc[i]
    subject = row["ticket_subject"]
    body = row["ticket_body"]
    query = f"subject : {subject}\n body : {body}"

    dept = row["department"]
    priority = row["priority"]

    dept_lookup[dept] = query
    priority_lookup[query] = priority

for i in [0,1,4,6]:
    row = df.iloc[i]
    subject = row["ticket_subject"]
    body = row["ticket_body"]
    query = f"subject : {subject}\n body : {body}"
    _type = row["type"]
    type_lookup[query] = _type

queries = []

for d,q in dept_lookup.items():
    query = f"{q} --> {d}"
    queries.append(query)

priority_queries = []
for d,q in priority_lookup.items():
    query = f"{d} --> {q}"
    priority_queries.append(query)

type_queries = []
for d,q in type_lookup.items():
    query = f"{d} --> {q}"
    type_queries.append(query)

In [41]:
type_queries

['subject : Discrepancia de facturación en Google Workspace\n body : Estimado equipo de soporte de TI,\n\nEstoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta.\n\nSaludos,\n\n<name> --> Incident',
 'subject : Urgent Consultation Request for Critical IT Issues\n body : Dear IT Services Support Team, I hope this message finds you well. My name is <name> and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please contact us at your earliest convenience, either via email or phone at <tel_num>. The swift resol